<a href="https://colab.research.google.com/github/BalavSha/Deep-Learning/blob/main/Classification_using_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>**Classification using Transfer Learning**</center>

### 1. Download the Dataset

In [1]:
# Import necessary libraries
import os
import urllib.request
import tarfile

In [2]:
# Define the URL and filename of the dataset
url = 'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz'
filename = '102flowers.tgz'

# Create a directory to store the dataset
os.makedirs('102flowers', exist_ok=True)

In [3]:
# Download the dataset from the URL and save it to the 102flowers directory
urllib.request.urlretrieve(url, os.path.join('102flowers', filename))

# Extract the dataset from the downloaded tar file
with tarfile.open(os.path.join('102flowers', filename), 'r:gz') as tar:
    tar.extractall('102flowers')

### 2. Upload the Dataset to the Google Drive

In [4]:
# Mount Google Drive in Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Define the path to the project folder in Google Drive
project_folder = '/content/drive/MyDrive/102flowers'

# Copy the 102flowers folder to the project folder
!cp -r 102flowers "$project_folder"

### 3. Load and Preprocess the Dataset using Tensorflow

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the path to the project folder in Google Drive
project_folder = '/content/drive/MyDrive/102flowers'

# Define the batch size and image size
batch_size = 32
img_size = (224, 224)

*Since the Dataset is not organized into sub-folder as training and test sets:*

In [11]:
# Use the ImageDataGenerator class to create data generators for the train and validation sets
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [12]:
# create generators for the train and validation sets
train_generator = datagen.flow_from_directory(
    os.path.join(project_folder, '102flowers'),
    target_size=img_size,
    batch_size=batch_size,
    shuffle=True,
    subset='training'
)

val_generator = datagen.flow_from_directory(
    os.path.join(project_folder, '102flowers'),
    target_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    subset='validation'
)

Found 6552 images belonging to 1 classes.
Found 1637 images belonging to 1 classes.


### 4. Define the Model using Transfer Learning

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

In [14]:
# Load the ResNet50V2 model with pre-trained weights
base_model = ResNet50V2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.summary()

94668760/94668760 [==============================] - 1s 0us/step
Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                        

In [15]:
# Freeze the weights of the backbone layers so that they are not updated during training phase
for layer in base_model.layers:
    layer.trainable = False

In [16]:
# define the number of output classes
num_classes = 102

# Replace the fully connected layer of the backbone with a new fully connected layer
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# create a new model by combining pre_trained model and new fully conntected layer for desired outputs
model = Model(inputs=base_model.input, outputs=predictions)

In [17]:
# display the architecture of newly custom model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

### 5. Compile the Model with appropriate Loss Funtion, Optimizers and Accuracy Metrics

In [18]:
# Compile the model with categorical cross-entropy loss, Adam optimizer, and accuracy metric
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### 6. Train the Model with training set

In [ ]:
# Define the number of epochs and steps per epoch for training
epochs = 10
steps_per_epoch = train_generator.n // batch_size

# Train the model on the training set
history = model.fit(train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch, validation_data=val_generator)

Epoch 1/10
204/204 [==============================] - 1716s 8s/step - loss: 9671.9355 - accuracy: 0.0021 - val_loss: 12472.7725 - val_accuracy: 0.0000e+00
Epoch 2/10
204/204 [==============================] - 1645s 8s/step - loss: 13179.2549 - accuracy: 0.0164 - val_loss: 14567.3789 - val_accuracy: 0.0000e+00
Epoch 3/10
204/204 [==============================] - 1622s 8s/step - loss: 14172.3604 - accuracy: 0.0221 - val_loss: 14074.8193 - val_accuracy: 0.0000e+00
Epoch 4/10
204/204 [==============================] - 1584s 8s/step - loss: 15080.9102 - accuracy: 0.0181 - val_loss: 17528.1328 - val_accuracy: 0.0000e+00
Epoch 5/10
204/204 [==============================] - 1575s 8s/step - loss: 20882.6758 - accuracy: 0.0140 - val_loss: 25627.1934 - val_accuracy: 0.0000e+00
Epoch 6/10
204/204 [==============================] - 1588s 8s/step - loss: 29568.1973 - accuracy: 0.0113 - val_loss: 32205.4766 - val_accuracy: 0.0000e+00
Epoch 7/10
204/204 [==============================] - ETA: 0s - l

### 7. Evaluate the trained with the validation set

In [ ]:
# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Acc: {val_acc} & Validation Loss: {val_loss}")

-------

### **8. Fine Tune the Hyper_parameters of the models if necessary:**<br>

*Unfreeze the last few layers of the backbone for fine-tuning*<br>
**for layer in base_model.layers[-30:]:** <br>
 ......**layer.trainable = True**<br><br>

*Lower the learning rate for fine-tuning*<br>
**optimizer = tf.keras.optimizers.Adam(lr=1e-5)**<br><br>

*Compile the model with categorical cross-entropy loss, the new optimizer, and accuracy metric*<br>
**model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])**<br><br>

*Train the model on the training set with a smaller learning rate for fine-tuning*<br>
**history = model.fit(train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch, validation_data=val_generator)**<br><br>

*Evaluate the model on the validation set*<br>
**val_loss, val_acc = model.evaluate(val_generator)**<br>
**print('Validation accuracy:', val_acc)**

-------

### 9. Save the Trained Model for future use

In [ ]:
# Save the trained model to a file
model.save('flower_classification_model.h5')

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

### 10. Load the Saved Model for use

In [ ]:
# Load the saved model from the file
model = load_model('flower_classification_model.h5')

### 11. Make predictions on new data

In [ ]:
# Define the path to the new image
new_image_path = '/path/to/new/image.jpg'

# Preprocess the new image as before with training images
img = image.load_img(new_image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.

In [ ]:
# Use the model to make predictions on the new image
predictions = model.predict(img_array)
class_index = np.argmax(predictions)

### 12. Evaluate the Model on the test data

In [ ]:
# Load the saved model from the file
model = load_model('flower_classification_model.h5')

In [ ]:
# Define the path to the test directory
test_dir = '/path/to/test/directory'

# Use the ImageDataGenerator class to create a generator for the test set
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

# <center>... The End ...</center>